# Analyze DTI

In [1]:
# import packages
# -----------------------------------------------------------------------------
# import pip
# pip.main(['install', 'seaborn']) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import subprocess
import sys
import json
import platform

In [2]:
# parameters
# -----------------------------------------------------------------------------
running_on = 'server' if 'Linux' in platform.system() else 'my_mac'

if running_on == 'my_mac':
    data_path = '/Users/ranigera/Dropbox/DTI_tests'
    preproc_path = '/Users/ranigera/Dropbox/DTI_tests/preproc'
    launch_files_path = '/Users/ranigera/Dropbox/DTI_analysis/launch_files'
else:
    data_path = '/export2/DATA/HIS/HIS_server/BIDS'
    preproc_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc'
    launch_files_path = '/export2/DATA/HIS/HIS_server/codes_dwi/launch_files'

expectedVolums = {
    'AP': 69,
    'PA' : 7,
    }
expectedB0s_indxs = {
    'AP_before': [0, 1, 18, 35, 52],
    'PA_before': [0, 2, 3, 4, 5, 6],
    'AP_after': [0, 1, 18, 35, 52],
    'PA_after': [0, 2, 3, 4, 5, 6]
    }

n_cores_TOPUP = 4

EDDY_command = 'eddy_openmp' if running_on == 'server' else 'eddy'
n_expected_EDDY_output_files = 13
n_cores_EDDY = 4

In [3]:
# Get folders and remove excluded subjects
# -----------------------------------------------------------------------------
print('>> Get sub folders')
subjFolders = [el for el in os.listdir(data_path) if 'sub' in el]

if running_on == 'my_mac':
    print('>> Get exclusion list')
    with open('/Users/ranigera/Google_Drive_TAU/Experiments/HIS_STUDY/Analysis/codes/paths_and_vars.py') as txtFile:
        txt = txtFile.read()
    participantsToExclude = [int(el) for el in txt.split('participantsToExclude = [')[1].split(']')[0].replace('\n','').replace('\n','').replace("'","").split(',')]

    print('>> Remove sub folders of excluded participants in case they are there')
    subjFolders = [el for el in subjFolders if int(el.split('-')[1]) not in participantsToExclude]

subjFolders.sort()

>> Get sub folders
>> Get exclusion list
>> Remove sub folders of excluded participants in case they are there


## Check for missing scans or wrong phase encoding directions for ALL SUBJECTS 

In [4]:
print ('>> Verify that all the scans exist and that the phase encoding directions are as they should.')
subjectsWithAProblem = []
for subjFolder in subjFolders:
    sub = int(subjFolder.split("-",1)[1])
    group = '1day' if sub < 200 else '3day'
    last_sess = group[0]
    DWI_path_before = os.path.join(data_path, subjFolder, 'ses-1/dwi/')
    DWI_path_after = os.path.join(data_path, subjFolder, f'ses-{last_sess}/dwi/')
    scansBaseNames = {
        'AP_before': f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-ap_run-01_dwi")}',
        'PA_before' : f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-pa_run-01_dwi")}',
        'AP_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-ap_run-02_dwi")}',
        'PA_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-pa_run-02_dwi")}'
        }

    for scan in scansBaseNames.keys():
        # print(scansBaseNames[scan] + '.json')
        # print(scanData['PhaseEncodingDirection'])
        if not os.path.exists(scansBaseNames[scan] + '.json'):
            subjectsWithAProblem.append(sub)
            print(' *** Scan not found: ' + scansBaseNames[scan] + '.json')
            continue
        with open(scansBaseNames[scan] + '.json') as json_file:        
            scanData = json.load(json_file)
            if ('acq-ap_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j-') or \
                ('acq-pa_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j'):
                subjectsWithAProblem.append(sub)
                print(' *** There is a problem with the scanning directions: ' + scansBaseNames[scan] + '.json is defined as ' + scanData['PhaseEncodingDirection'] + '.')
                continue

subjectsWithAProblem = list(set(subjectsWithAProblem))
subjectsWithAProblem.sort()

>> Verify that all the scans exist and that the phase encoding directions are as they should.


# [TEMP] Working on subject (for now)

## Get b0 volume indices and perform bval & bvec QA

In [5]:

print('> Verify that data points in the bval files is as expected.')
if pd.read_csv(scansBaseNames['AP_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['PA'] or \
    pd.read_csv(scansBaseNames['AP_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['PA']:
        print(f' *** The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')
        raise Exception(f'The number of data points in the bval for one of the scans for subjetc {sub} is not as expected.')

print('> Verify that data points in the bvec files is as expected.')
if pd.read_csv(scansBaseNames['AP_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['PA'] or \
    pd.read_csv(scansBaseNames['AP_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
    pd.read_csv(scansBaseNames['PA_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['PA']:
        print(f' *** The number of data points in the bvec for one of the scans for subjetc {sub} is not as expected.')
        raise ValueError(f'The number of data points in the bvec for one of the scans for subjetc ' + str(sub) + ' is not as expected.')


print('> Extract B0s:')
B0s_indxs = {}
for scan in scansBaseNames.keys():
    B0s=pd.read_csv(scansBaseNames[scan] + '.bval', header=None, sep=' ').T
    B0s.columns = ['bval']
    B0s_indxs[scan] = list(B0s[B0s.bval < 20].index)

print('> Verify that b0 quantity and indices are as expected.')
if B0s_indxs != expectedB0s_indxs:
    print(f' *** The indices of the b0s for one of the scans for subjetc {sub} are not as expected.')
    raise ValueError(f'The indices of the b0s for one of the scans for subjetc ' + str(sub) + ' are not as expected.')


> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.


## Preprocessing

#### Create pre-processing folder

In [6]:
print('> Create preprocessing folder')
try:
    os.makedirs(os.path.join(preproc_path, subjFolder), exist_ok=False)
except:
    pass

> Create preprocessing folder


### (1) TOPUP
* A tool for estimating and correcting susceptibility induced distortions

#### Extract B0 volumes

In [7]:
print('> Extract B0s (using the fslroi):')
sub_B0s_files = []
for scan in scansBaseNames.keys():
    for B0ind in B0s_indxs[scan]:
        B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
        sub_B0s_files.append(B0_file_name)
        if not os.path.isfile(f"{B0_file_name}"):
            print(f'>> runs: fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
            os.system(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
print(f">> COMPLETED.")

> Extract B0s (using the fslroi):
>> COMPLETED.


#### Merge B0 volumes for TOPUP

In [13]:
print('> Merge B0s for each session (using the fslmerge):')
#print([scan for scan in sub_B0s_files if 'AP_before' in scan])
#print([scan for scan in sub_B0s_files if 'PA_before' in scan])
for time in ['before', 'after']:
    output_base_name = f'{os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s'
    if not os.path.isfile(f"{output_base_name}.nii.gz"):
        print(f'>> runs: fslmerge -t {output_base_name}' + ' ' + ' '.join([scan for scan in sub_B0s_files if f'AP_{time}' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if f'PA_{time}' in scan]))
        os.system(f'fslmerge -t {output_base_name}' + ' ' + ' '.join([scan for scan in sub_B0s_files if f'AP_{time}' in scan]) + ' ' + ' '.join([scan for scan in sub_B0s_files if f'PA_{time}' in scan]))
print(f">> COMPLETED.")

> Merge B0s for each session (using the fslmerge):
>> COMPLETED.


#### Generate the acqparams.txt files

In [14]:
print('> Create the acqparams.txt files (one per session [''before'' and ''after''])')
# first get the totalReadoutTime from the json files
total_readout_time = {}
for scan in scansBaseNames.keys():
    with open(scansBaseNames[scan] + '.json') as json_file:        
        scanData = json.load(json_file)
        total_readout_time[scan] = scanData['TotalReadoutTime']

for time in ['before', 'after']:
    acqPars=[f'0 -1 0 {total_readout_time[f"AP_{time}"]}' for scan in sub_B0s_files if f'AP_{time}' in scan] + [f'0 1 0 {total_readout_time[f"PA_{time}"]}' for scan in sub_B0s_files if f'PA_{time}' in scan]
    # write a list of strings to a file (one string per line):
    with open(os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt'), 'w') as f:
        for item in acqPars:
            f.write("%s\n" % item)

> Create the acqparams.txt files (one per session [before and after])


#### Run TOPUP

Create TOPUP launch files

In [129]:
print('> Create the TOPUP launch files (one per subject [''before'' and ''after''] together)')

launch_file = os.path.join(launch_files_path, 'TOPUP_' + subjFolder + '_launch.txt')

if not os.path.isfile(launch_file):
    with open(launch_file, 'w') as f:
        for time in ['before', 'after']:
                print(f">> This command was written to {launch_file}:\ntopup --imain={os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s \\\n\
                        --datain={os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt')} \\\n\
                        --config=b02b0.cnf \\\n\
                        --out={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
                        --iout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout \\\n\
                        --fout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout \n\
                    ")
                f.write(f"topup --imain={os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s --datain={os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt')} --config=b02b0.cnf --out={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s --iout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout --fout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout\n")
print(f">> COMPLETED.")

> Create the TOPUP launch files (one per subject [before and after] together)
>> COMPLETED.


Run TOPUP launch files

In [136]:
print('> Run the TOPUP launch files (one per subject [''before'' and ''after''] together)')

launch_file = os.path.join(launch_files_path, 'TOPUP_' + subjFolder + '_launch.txt')

# check if all expected output files exist:
expected_TOPUP_out_files_exist = []
for time in ['before', 'after']:
    expected_TOPUP_out_files_exist += [os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fieldcoef.nii.gz"),
                                    os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout.nii.gz"),
                                    os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout.nii.gz"),
                                    os.path.isfile(f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_movpar.txt")]
 
 # Run the launch file if not all output files present:
if not all(expected_TOPUP_out_files_exist):
            print(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_TOPUP} -r inf")
            #os.system(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_TOPUP} -r inf")
            
print(f">> COMPLETED.")

> Run the TOPUP launch files (one per subject [before and after] together)
>> COMPLETED.


### (2) EDDY
* A tool for correcting Eddy currents (and motion)

#### Average TOPUP corrected (unwarped) B0 volums (the .iout file) [for now it prints the command]

In [79]:
for time in ['before', 'after']:
      output_file_name = f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg.nii.gz"
      if not os.path.isfile(f"{output_file_name}"):
            print(f">> runs: fslmaths {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout -Tmean {output_file_name}")
            os.system(f"fslmaths {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout -Tmean {output_file_name}")
print(f">> COMPLETED.")

>> COMPLETED.


#### Run BET on the averaged B0s volume

In [80]:
for time in ['before', 'after']:
      output_base_name = f"{os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain"
      if not os.path.isfile(f"{output_base_name}.nii.gz") or not os.path.isfile(f"{output_base_name}_mask.nii.gz"):
            print(f">> runs: bet {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg {output_base_name} -m -f 0.2")
            os.system(f"bet {os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg {output_base_name} -m -f 0.2")
print(f">> COMPLETED.")

>> COMPLETED.


#### Create index.txt file
This file maps the volumes in the main DTI data to the relevant line in the acqparams.txt and in the movpar.txt (assessed movement parameters from TOPUP) files
* details in: https://www.youtube.com/watch?v=1T1cRnX7MpA

In [81]:
# create an index.txt file for the EDDY
for time in ['before', 'after']:
    ind_to_write = 1
    with open(os.path.join(preproc_path, subjFolder, f'{time}_index.txt'), 'w') as f:
        for i in range(expectedVolums['AP']):   
            if i > 0 and i in expectedB0s_indxs[f'AP_{time}']:
                ind_to_write += 1
            f.write("%s\n" %ind_to_write)
print('>> COMPLETED.')


>> COMPLETED.


#### Run EDDY [currently printing the command]
* Correct for eddy currents and subject movement (and taking to account the suceptibility field calculated by TOPUP)

In [82]:
for time in ['before', 'after']:
     print(f"eddy_openmp --imain={scansBaseNames[f'AP_{time}']}.nii.gz \\\n\
          --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask \\\n\
          --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')}\\\n\
          --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} \\\n\
          --bvecs={scansBaseNames[f'AP_{time}']}.bvec \\\n\
          --bvals={scansBaseNames[f'AP_{time}']}.bval \\\n\
          --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
          --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} \\\n\
          --verbose \
     ")



eddy_openmp --imain=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.nii.gz \
          --mask=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s_iout_avg_brain_mask \
          --index=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/before_index.txt\
          --acqp=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/sub-120_before_acqparams.txt \
          --bvecs=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.bvec \
          --bvals=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.bval \
          --topup=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/topup_sub-120_AP_PA_before_b0s \
          --out=/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/eddy_unwarped_images_sub-120_before \
          --verbose      
eddy_openmp --imain=/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-02_dwi.nii.gz \
          --mask=/Use

Create EDDY launch files

In [153]:
print('> Create the EDDY launch files (one per subject [''before'' and ''after''] together)')

launch_file = os.path.join(launch_files_path, 'EDDY_' + subjFolder + '_launch.txt')

if not os.path.isfile(launch_file):
    with open(launch_file, 'w') as f:
        for time in ['before', 'after']:
            # print(f">> This command was written to {launch_file}:\ntopup --imain={os.path.join(preproc_path, subjFolder,subjFolder)}_AP_PA_{time}_b0s \\\n\
            #         --datain={os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt')} \\\n\
            #         --config=b02b0.cnf \\\n\
            #         --out={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
            #         --iout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout \\\n\
            #         --fout={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_fout \n\
            #     ")
            print(f">> This command was written to {launch_file}:\n{EDDY_command} --imain={scansBaseNames[f'AP_{time}']}.nii.gz \\\n\
                --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask \\\n\
                --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')}\\\n\
                --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} \\\n\
                --bvecs={scansBaseNames[f'AP_{time}']}.bvec \\\n\
                --bvals={scansBaseNames[f'AP_{time}']}.bval \\\n\
                --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s \\\n\
                --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} \\\n\
                --verbose \n\
            ")
            f.write(f"{EDDY_command} --imain={scansBaseNames[f'AP_{time}']}.nii.gz --mask={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s_iout_avg_brain_mask --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')} --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} --bvecs={scansBaseNames[f'AP_{time}']}.bvec --bvals={scansBaseNames[f'AP_{time}']}.bval --topup={os.path.join(preproc_path, subjFolder,'topup_' + subjFolder)}_AP_PA_{time}_b0s --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} --verbose\n")
print(f">> COMPLETED.")

> Create the EDDY launch files (one per subject [before and after] together)
>> COMPLETED.


Run EDDY launch files

In [150]:
print('> Run the EDDY launch files (one per subject [''before'' and ''after''] together)')

launch_file = os.path.join(launch_files_path, 'EDDY_' + subjFolder + '_launch.txt')

# check if all expected output files exist:
expected_EDDY_out_files_exist = []
for time in ['before', 'after']:
    eddy_output_files = glob.glob(os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}*')) # get files in a directory
    expected_EDDY_out_files_exist.append(len(list(set(eddy_output_files))) == n_expected_EDDY_output_files) # check that the number of unique EDDY files is as expected

 # Run the launch file if not all output files present:
if not all(expected_EDDY_out_files_exist):
            print(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_EDDY} -r inf")
            os.system(f">> Running: launch -s {launch_file} -j schonberglab -p {n_cores_EDDY} -r inf")
print(f">> COMPLETED.")

> Run the EDDY launch files (one per subject [before and after] together)
>> Running: launch -s /Users/ranigera/Dropbox/DTI_analysis/launch_files/EDDY_sub-120_launch.txt -j schonberglab -p 4 -r inf
>> COMPLETED.


#### Run EDDY QC

In [5]:
subjFolder

'sub-120'

In [13]:
{os.path.join(preproc_path, subjFolder, f'{time}_index.txt')}

{'/Users/ranigera/Dropbox/DTI_tests/preproc/sub-120/before_index.txt'}

In [26]:
print('weew\nwerewr')

weew
werewr


In [39]:
for time in ['before', 'after']:
    os.system(f"eddy_quad {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \
    -idx {os.path.join(preproc_path, subjFolder, f'{time}_index.txt')} \
    -par {os.path.join(preproc_path, subjFolder, subjFolder  + f'_{time}_acqparams.txt')} \
    -m   {os.path.join(preproc_path, subjFolder, f'topup_{subjFolder}_AP_PA_{time}_b0s_iout_avg_brain_mask.nii.gz')} \
    -b   {scansBaseNames[f'AP_{time}'] + '.bval'}")

#f{os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} -idx {time}_index.txt -par sub-120_before_acqparams.txt -m topup_sub-120_AP_PA_before_b0s_iout_avg_brain_mask.nii.gz -b /Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.bval

In [35]:
scansBaseNames[f'AP_{time}'] + '.bval'

'/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi.bval'

In [34]:
scansBaseNames[f'AP_{time}']

'/Users/ranigera/Dropbox/DTI_tests/sub-120/ses-1/dwi/sub-120_ses-1_acq-ap_run-01_dwi'